In [2]:
from dhanhq import dhanhq
import yaml
import pandas  as pd
import numpy as np


In [3]:

## Authentication
CONFIG_FILE_PATH = r"..\\configs\\config.yaml"
with open(CONFIG_FILE_PATH,'r') as config:
    api_config = yaml.safe_load(config)
APPLICAITON_ID = api_config.get('api_config')[0]
CLIENT_ID = api_config.get('api_config')[1]
ACCESS_TOKEN = api_config.get('api_config')[2]
EXP_HOURS = api_config.get('api_config')[3]

dhan = dhanhq(CLIENT_ID,ACCESS_TOKEN)


In [9]:
k = [(5, '435823', 15), (0, '13', 15), (0, '25', 15), (5, '439850', 15), (5, '439849', 15), (5, '439848', 15), (5, '439847', 15), (5, '439846', 15), (5, '439845', 15), (5, '439844', 15), (5, '439843', 15), (5, '439842', 15), (5, '439841', 15), (5, '439840', 15), (5, '439839', 15), (5, '439838', 15), (5, '439833', 15), (5, '439863', 15), (5, '439864', 15), (5, '439865', 15), (5, '439866', 15), (5, '439867', 15), (5, '439868', 15), (5, '439869', 15)]

In [11]:
k[3][1]

'439850'

['439850',
 '439849',
 '439848',
 '439847',
 '439846',
 '439845',
 '439844',
 '439843',
 '439842',
 '439841',
 '439840',
 '439839',
 '439838',
 '439833',
 '439863',
 '439864',
 '439865',
 '439866',
 '439867',
 '439868',
 '439869']

In [8]:
dhan.expiry_list(435823,"MCX_COMM")['data']['data']

['2024-12-16', '2025-01-15', '2025-02-17']

In [15]:

## Loading other configuration 
OTHER_CONFIG_FILE_PATH = r"..\\configs\\other_config.yaml" # This we will configure all other variables
with open(OTHER_CONFIG_FILE_PATH,'r') as other_configs:
    gen_config = yaml.safe_load(other_configs)
underlying = gen_config.get('index_file_metadata')
options = gen_config.get('options_file_metadata')


In [16]:

# functions
def option_age(x): #this function will classify the options strike if it belongs to Curent series, Next series or Far next.
    if x == 1.0:
        return 'C' # Current series
    elif x == 2.0 :
        return 'N' # Next Seroes
    else :
        return 'F' # Far next series



In [17]:
# Load the latest keys and sripts metadata automatically
security_list = dhan.fetch_security_list("compact")


d:\Miniconda\envs\dhan\Lib\site-packages\dhanhq\dhanhq.py:1053: DtypeWarning: Columns (14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filename)


In [18]:

# Filter for only NIFTY and BANKNIFTY OPTIONS and MCX
#---------------------------------------------
NSE=security_list[(
(security_list['SEM_EXM_EXCH_ID'] == 'NSE') & 
(security_list['SEM_INSTRUMENT_NAME'] == 'OPTIDX') & 
((security_list['SEM_TRADING_SYMBOL'].str.startswith('BANKNIFTY')) | (security_list['SEM_TRADING_SYMBOL'].str.startswith('NIFTY')))
&
(security_list['SEM_TRADING_SYMBOL'].str.startswith('NIFTYNXT50')==False)
)]

MCX= security_list[(security_list['SEM_EXM_EXCH_ID'] == 'MCX') & (security_list['SEM_INSTRUMENT_NAME'] == 'OPTFUT') & (security_list['SM_SYMBOL_NAME']=='CRUDEOIL')]

filtered_df = pd.concat([NSE,MCX])

filtered_df = filtered_df.copy() # We are copying as a new instance to avoid slice warning


In [19]:
# Below logic will help us to classify the Series type
filtered_df['SEM_EXPIRY_DATE_CUSTOM'] = np.nan
filtered_df['SEM_EXPIRY_DATE_CUSTOM'] = pd.to_datetime(filtered_df.SEM_EXPIRY_DATE)
filtered_df['SEM_UNDERLYING'] = [x[0] for x in filtered_df.SEM_CUSTOM_SYMBOL.str.split(' ')]
filtered_df['Rank'] = filtered_df.groupby('SEM_UNDERLYING')['SEM_EXPIRY_DATE_CUSTOM'].rank(method='dense', ascending=True)

filtered_df['Series'] = filtered_df['Rank'].apply(option_age)



In [20]:
#for MCX we are restricting to current month contract only due to liquidity
filtered_df = filtered_df[((filtered_df['SEM_EXM_EXCH_ID']=='MCX') & (filtered_df['Series']=='C')) | (filtered_df.Series != 'F')] # We are removing the Farnext sereies due to better memory management
filtered_df[['SEM_SMST_SECURITY_ID','SEM_LOT_UNITS','SEM_CUSTOM_SYMBOL','SEM_STRIKE_PRICE','SEM_OPTION_TYPE','SEM_UNDERLYING','Series']].to_excel(options,index=False)

In [21]:
#------------------------------------
# index
#------------------------------------
mcx_fut =  security_list[(security_list['SEM_EXM_EXCH_ID']=='MCX') & (security_list['SM_SYMBOL_NAME']=='CRUDEOIL') & (security_list['SEM_INSTRUMENT_NAME']=='FUTCOM')]
index = security_list[(security_list['SEM_INSTRUMENT_NAME'] == 'INDEX') & (
    (security_list['SEM_TRADING_SYMBOL']=='BANKNIFTY')
    |
    (security_list['SEM_TRADING_SYMBOL']=='NIFTY')
)
&
(security_list['SEM_SEGMENT']=='I')]
filtered_df_index = pd.concat([mcx_fut,index])
filtered_df_index = filtered_df_index.copy()
filtered_df_index['SEM_TRADING_SYMBOL']= [x[0] for x in filtered_df_index.SEM_TRADING_SYMBOL.str.split('-')]

In [22]:
filtered_df_index['Rank'] = filtered_df_index.groupby('SEM_EXM_EXCH_ID')['SEM_EXPIRY_DATE'].rank(method='dense', ascending=True)
filtered_df_index = filtered_df_index[(filtered_df_index['Rank']==1.0) | (np.isnan(filtered_df_index['Rank']))]
filtered_df_index[['SEM_SMST_SECURITY_ID','SEM_TRADING_SYMBOL']].to_excel(underlying,index=False)